# Training a network

Let's take a detailed look at how we train a new deep learning model from pre-prepared data with dtoolAI. We're going to train a convolutional neural network (CNN) to recognise handwritten digits from the MNIST dataset.

## Loading our data

First, we'll need to load our data, using the appropriate class:

In [1]:
from dtoolai.data import TensorDataSet

We can now load our data from a persistent identifier (URI):

In [2]:
train_dataset_uri = "http://bit.ly/2uqXxrk"
train_ds = TensorDataSet(train_dataset_uri)

Let's look at what we've loaded:

In [3]:
data, label = train_ds[9000]
print(data.shape)
print(label)

(1, 28, 28)
1


The data are 1x28x28 arrays representing digits. We can load a helper function to visualise them:

In [4]:
from dtoolai.data import scaled_float_array_to_pil_image

Then visualise a single digit like this:

In [5]:
scaled_float_array_to_pil_image(data)

## Setting parameters

Next we'll set some parameters with which to train our model. We'll use dtoolAI's Parameters class for this, since we'll then be able to record those parameters during training automatically.

In [6]:
from dtoolai.parameters import Parameters

In [7]:
params = Parameters(
    batch_size=128,
    learning_rate=0.01,
    n_epochs=1
)

## Model, optimiser, loss function

In general, to train a deep learning model we need three things:

1. A suitable model architecture.
2. A loss function (how the difference between target labels and predicted labels will be calculated).
3. Training data.

We've loaded the data, now we need the model and a loss function. We'll also need to choose an optimiser.

Firstly, we'll load our generic classifier model:

In [8]:
from dtoolai.models import GenNet

then we can set the model's parameters from what we know about the dataset and initialise it:

In [9]:
params['init_params'] = dict(input_channels=train_ds.input_channels, input_dim=train_ds.dim)
model = GenNet(**params['init_params'])

Now we can create a loss function and optimiser:

In [10]:
import torch
loss_fn = torch.nn.NLLLoss()
optim = torch.optim.SGD(model.parameters(), lr=params.learning_rate)

## Training

Now we're ready to train our model. First we import a helper function from dtoolAI:

In [11]:
from dtoolai.training import train_model_with_metadata_capture

and another function to make sure our new model is packaged with useful metadata:

In [12]:
from dtoolcore import DerivedDataSetCreator

Now we can train the model:

In [14]:
with DerivedDataSetCreator('mnist.example.model', '../scratch', train_ds) as output_ds:
    train_model_with_metadata_capture(model, train_ds, optim, loss_fn, params, output_ds)

Using device cpu
  Epoch 0, batch 0/469, running loss 2.306
  Epoch 0, batch 10/469, running loss 25.385
  Epoch 0, batch 20/469, running loss 48.437
  Epoch 0, batch 30/469, running loss 71.473
  Epoch 0, batch 40/469, running loss 94.530
  Epoch 0, batch 50/469, running loss 117.592
  Epoch 0, batch 60/469, running loss 140.637
  Epoch 0, batch 70/469, running loss 163.678
  Epoch 0, batch 80/469, running loss 186.687
  Epoch 0, batch 90/469, running loss 209.687
  Epoch 0, batch 100/469, running loss 232.699
  Epoch 0, batch 110/469, running loss 255.694
  Epoch 0, batch 120/469, running loss 278.677
  Epoch 0, batch 130/469, running loss 301.655
  Epoch 0, batch 140/469, running loss 324.613
  Epoch 0, batch 150/469, running loss 347.553
  Epoch 0, batch 160/469, running loss 370.473
  Epoch 0, batch 170/469, running loss 393.384
  Epoch 0, batch 180/469, running loss 416.292
  Epoch 0, batch 190/469, running loss 439.195
  Epoch 0, batch 200/469, running loss 462.075
  Epoch 0, ba

## Evaluating the model

The MNIST dataset actually contains 60,000 training examples (which we used to train our model), and 10,000 test examples. We can use this latter set of examples to test our model.

In [15]:
mnist_test_uri = "http://bit.ly/2NVFGQd"
test_ds = TensorDataSet(mnist_test_uri)

In [29]:
from torch.utils.data import DataLoader

In [40]:
test_dl = DataLoader(test_ds, batch_size=4)

In [31]:
from dtoolai.utils import evaluate_model

In [41]:
correct = evaluate_model(model, test_dl)

In [43]:
correct

9542

## Checking model provenance

In [16]:
from dtoolai.trained import TrainedTorchModel

In [17]:
ttm = TrainedTorchModel("../scratch/mnist.example.model/")

In [19]:
ttm.dataset.get_annotation("source_dataset_uri")

'http://bit.ly/2uqXxrk'

In [20]:
import dtoolcore

In [21]:
source_dataset = dtoolcore.DataSet.from_uri('http://bit.ly/2uqXxrk')

In [23]:
print(source_dataset.get_readme_content())

---
dataset_name: MNIST handwritten digits
project: dtoolAI demonstration datasets
authors:
  - Yann LeCun
  - Corinna Cortes
  - Christopher J.C. Burges
origin: http://yann.lecun.com/exdb/mnist/
usetype: train

